# Detailed Evaluation of Hand-Tuned algorithm

Versions and contact same as in siblings_ml.ipynb

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from IPython.display import Image  
import pandas
import seaborn
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def get_pd_files(folder):
    sibf = folder + "hosts.csvcapture.pcap.ts.siblingresult.csv"
    nonsibf = folder + "hosts.csv__nonsiblings_seed1_n*capture.pcap.ts.siblingresult.csv"
    import glob
    for filename in glob.glob(nonsibf):
        nonsibf = filename
    import os.path
    if os.path.isfile(sibf) and os.path.isfile(nonsibf):
        print("Loading from filenames {} and {}".format(sibf, nonsibf))
    else:
        print("Files not found {} and {}".format(sibf, nonsibf))
        
    sib = pd.read_csv(sibf, index_col=0)
    sib['label'] = 1
    nonsib = pd.read_csv(nonsibf, index_col=0)
    nonsib['label'] = 0
    print("Read {} siblings and {} non-siblings from files.".format(len(sib), len(nonsib)))
    return sib, nonsib
       
def dec2prd_ours(df):
    df.loc[df["decision"].str.contains("^sibling"), "dec_prd"] =  1
    df.loc[df["decision"].str.contains("^non-sibling"), "dec_prd"] =  0
    return  # df is changed in place so no returning necessary

def dec2prd_bev(df):
    df.loc[df["dec_bev"].str.contains("^sibling"), "dec_bev_prd"] =  1
    df.loc[df["dec_bev"].str.contains("^non-sibling"), "dec_bev_prd"] =  0
    return  # df is changed in place so no returning necessary

def mix_sib_nonsib(sib, nonsib, mode, rs=42):
    if mode == "equal":
        nonsibint = nonsib.sample(n=len(sib), replace=True, weights=None, random_state=rs)
    else:
        nonsibint = nonsib
    datain = pd.concat([sib,nonsibint])
    #print("merged shape: {}".format(datain.shape))
    #print("columns: {}".format(list(datain.columns.values)))
    return datain


def get_ouralgo_stats(sib, nonsib):
    #print("Our algo stats:")
    df = mix_sib_nonsib(sib, nonsib, "full", 42)
    df_ours = df[["label", "decision"]].copy()
    #print(df_ours[df_ours.isnull().any(axis=1)])
    dec2prd_ours(df_ours)
    undec = len(df_ours[df_ours.isnull().any(axis=1)])
    print("Our algo: Not deciding on {} pairs for unknown/error reasons.".format(undec))
    df_ours = df_ours.dropna()
    weights = get_sample_weight_one_input(df_ours)
    mcc = matthews_corrcoef(df_ours["label"], df_ours["dec_prd"], sample_weight=None)
    f1 = f1_score(df_ours["label"], df_ours["dec_prd"], sample_weight=None)
    print("Our algo stats: ({}) undecided, mcc: {}, f1: {}".format(undec, mcc, f1))
    statsv = list(stats(df_ours["label"], df_ours["dec_prd"]))
    #prec = precision_score(df_ours["label"], df ,_ours["dec_prd"], sample_weight=None)
    statsv.append(mcc)
    return statsv

def get_bev_stats(sib, nonsib):
    #print("Beverly algo stats:")
    df = mix_sib_nonsib(sib, nonsib, "full", 42)
    #print(df[df.isnull().any(axis=1)])
    df_tmp = df[["label", "dec_bev"]].copy()
    dec_nan = len(df_tmp[df_tmp["dec_bev"].isnull() == True])
    df_tmp = df_tmp[df_tmp["dec_bev"].isnull() == False]
    dec2prd_bev(df_tmp)
    undec = len(df_tmp[df_tmp.isnull().any(axis=1)])
    df_tmp = df_tmp.dropna()
    weights = get_sample_weight_one_input(df_tmp)
    mcc = matthews_corrcoef(df_tmp["label"], df_tmp["dec_bev_prd"], sample_weight=None)
    f1 = f1_score(df_tmp["label"], df_tmp["dec_bev_prd"], sample_weight=None)
    print("Beverly algo: Not deciding on {} pairs for NaN and {} pairs for unknown/error reasons.".format(dec_nan, undec))
    print("Beverly algo stats: ({}) undecided, mcc: {}, f1: {}".format(undec, mcc, f1))
    statsv =  list(stats(df_tmp["label"], df_tmp["dec_bev_prd"]))
    statsv.append(mcc)
    return statsv
    

def match_nonsibs_slow(sib, nonsib, rs=42):
    ctr = 0 
    for i, ii in sib.iterrows():
        for j, jj in sib.iterrows():
            if ii[1] != jj[1]:
                nscand = ii[1] + "_+_" +  jj[1]
                #print(nscand)
                ctr += 1 
                #if not (nonsib["domain"] == nscand).any():
                #   print("fail for {}".format(nscand))
                #  return
        #print(ctr)
    return
                
def match_nonsibs(sib, nonsib, rs=42):
    ctr = 0 
    a = []
    sd = dict() # siblings dict
    nsd = dict()  # non siblings dict
    for i in sib.itertuples():
        sd[i[0]] = 0
    for i in nonsib.itertuples():
        nsd[i[0]] = 0
    nscand = dict()
    #nstmp = pandas.DataFrame()
    for i in sd.keys():
        for j in sd.keys():
            if i != j:
                nscandstr = i + "_+_" +  j
                nscand[nscandstr] = 1
    print("Generated {} non-sibling candidates from {} siblings.".format(len(nscand), len(sib)))
    fails = []
    for k1 in nsd.keys():
        if k1 not in nscand.keys():
            fails.append(k1)
            #print("fail! {} ".format(i))    
    nsfiltered = nonsib.copy()
    nsfiltered.drop(fails, inplace=True)
    return nsfiltered

def assign_groups_old(datain):
    datain["group"] = "servers"
    datain.loc[datain["domain"].str.contains("nlnog.net"), "group"] = "nlnog"
    datain.loc[datain["domain"].str.contains("RA_"), "group"] = "RA"
    datain.loc[datain["domain"].str.extract("RA_([0-9]{4})") < 6019, "group"] = "RAv1"
    datain.loc[datain["domain"].str.extract("RA_([0-9]{4})") > 6018, "group"] = "RAv2"
    return
    #groups = datain["group"].as_matrix()
    #del datain["group"]
    #return groups

def assign_groups(datain):
    datain["group"] = "servers"
    #sib.loc[sib.index.str.contains("nlnog.net"), "group"] = "nlnog"
    datain.loc[datain.index.str.contains("nlnog.net"), "group"] = "nlnog"
    datain.loc[datain.index.str.contains("RA_"), "group"] = "RA"
    datain["ra_id"] = datain.index.str.extract("RA_([0-9]{4})", expand=False).astype(float).fillna(0).astype(int) 
    #datain.index.str.extract("RA_([0-9]{4})", expand=False).astype(float).fillna(0).astype(int) > 6018
    datain.loc[(datain.ra_id > 5999) & (datain.ra_id < 6019), "group"] = "RAv1"
    datain.loc[datain.ra_id > 6018, "group"] = "RAv2"    
    #datain.loc[datain.index.str.extract("RA_([0-9]{4})", expand=False) > 6018, "group"] = "RAv2"
    groups = datain["group"].as_matrix()
    return groups
    
    
def prune_datain(datain):
    errorc = len(datain[datain["decision"].str.contains("ERROR|error") == True])
    print("Removing {} errors values from datain.".format(errorc))
    datain = datain[datain["decision"].str.contains("ERROR|error") == False]

    hzdiffc = len(datain[datain["hzdiff"] != 0])
    print("Deciding {} hzdiff hosts as non-sib, stats:".format(hzdiffc))
    lbl = datain[datain["hzdiff"] != 0]["label"]
    prd = lbl.copy()
    prd[:] = 0
    stats(lbl,prd)
    dataout = datain[datain["hzdiff"] == 0]
    #  datain = datain[datain["domain"].str.contains("nlnog.net") == True]
    return dataout, lbl, prd

def prune_data_for_ml(datain):
    # just kick hzdiff out
    # problem: NaNs might be in non-feature columns such as RA_ID
    erridx = datain[datain.decision.str.contains("ERROR|error") == True].index
    labels, features = make_labels_features(datain)
    naidx = datain[features.isnull().any(axis=1) == True].index
    bothidx = erridx | naidx
    dataout = datain.drop(bothidx)
    #dataout = dataout[dataout.decision.str.contains("ERROR|error") == False]
    # TODO: should also calculcate stats on this!
    lbl = datain.loc[bothidx, "label"]
    prd = lbl.copy()
    prd[:] = 0
    stats(lbl,prd)    
    print("Removing {} rows with error results and {} rows with NaNs (typically hz different) from a \
    total of {} entries, resulting in {} entries.".format(
            len(erridx), len(naidx), len(datain), len(dataout)))
    return dataout, lbl, prd


def stats(lbl, prd):
        tp = np.sum((lbl == 1) & (prd == 1)) 
        fp = np.sum(lbl < prd ) 
        tn = np.sum((lbl == 0) & (prd == 0)) 
        fn = np.sum(lbl > prd ) 
        try:
            prec =  round(100*tp/(tp+fp),2) # TPR?
            recall = round(100*tp/(tp+fn),2) 
            spec= round(100*tn/(tn+fp),2) # TNR?
            acc = round(100*(tn+tp)/(tn+fn+fp+tp),2)
        except ZeroDivisionError as e:
            print("Catching ZeroDivisionError at stats!")
            prec = 0
            recall = 0
            spec = 0
            acc = 0
        print("Correct: {}, incorrect {}, TP {}, FP {}, TN {}, FN{}, Prec. {}, Rec. {}, Spec. {}, Acc. {}%".format(
        np.sum(lbl == prd),
        np.sum(lbl != prd),
        tp, fp, tn, fn, 
        prec, recall, spec, acc
        ))
        return prec, recall, spec, acc
        
def make_labels_features(dfin):
    labels = dfin["label"]
    features = dfin[["hzdiff", "hzr2diff", "timestamps_diff", "adiff", 
                        "theta", "r2diff", "ott_rng_diff_rel", "optsdiff",
                       "perc_85_val"]].copy()
    features["hzr2mean"] = (dfin["hz4r2"] + dfin["hz6r2"])  / 2.0
    features["r2mean"] = (dfin["r4_sqr"] + dfin["r6_sqr"]) / 2.0     
    features["ott_rng_mean"] = (dfin["ott4_rng"] + dfin["ott6_rng"]) / 2.0
    features["splinediff_scaled"] = dfin["perc_85_val"] / features["ott_rng_mean"]
    return labels, features   

def get_sample_weight(sib, nonsib):
    # WIP TODO
    #siblings = len(dfin[dfin["label"] == 1])
    #nonsiblings = len(datain[datain["label"] == 0])
    sl = len(sib)
    nsl = len(nonsib)
    tl = sl + nsl
    nsw = sl / tl
    sw = nsl / tl
    print("Found {} sibs and {} nonsibs, weights: {} and {}".format(sl, nsl, sw, nsw))
    weight = np.zeros(len(datain))
    weight = np.float32(datain["label"].as_matrix())
    weight[weight == 1] = sw
    weight[weight == 0] = nsw
    
    
def get_sample_weight_one_input(dfin):
    sl = len(dfin[dfin["label"] == 1])
    nsl = len(dfin[dfin["label"] == 0])
    tl = sl + nsl
    nsw = sl / tl
    sw = nsl / tl
    weight = np.zeros(len(dfin))
    weight = np.float32(dfin["label"].as_matrix())
    weight[weight == 1] = sw
    weight[weight == 0] = nsw
    print("Found {} sibs and {} nonsibs, weights: {} and {}, #weights: {}".format(
        sl, nsl, round(sw,4), round(nsw,4), len(weight)))
    return weight


# functions for ML with proprtional group sampling
def split_stratified_groups(sib, splits, nr):
    from sklearn.model_selection import KFold # non-overlapping!
    groups = assign_groups(sib)
    groupset = set(groups)
    gsibdf_train = pd.DataFrame(columns=sib.columns)
    gsibdf_test = pd.DataFrame(columns=sib.columns)
    for i in groupset:
        groupsib = sib[sib["group"] == i].copy()
        if len(groupsib ) < splits:
            # can not split into more folds than files...
            print("ERROR: more splits ({}) than samples ({}), reducing to sample nr".format(splits, len(groupsib)))
            splits = len(groupsib)
        #print("## GROUP: {} with {} elements.".format(i, len(groupsib)))
        ks = KFold(n_splits=splits, random_state=42, shuffle=True)
        labels, features = make_labels_features(groupsib)
        ctr = -1
        for train_index, test_index in ks.split(groupsib):
            ctr += 1                
            if (ctr == nr):
            #print("TRAIN:", train_index, "TEST:", test_index)
                gsibdf_train = gsibdf_train.append(groupsib.iloc[train_index])
                gsibdf_test = gsibdf_test.append(groupsib.iloc[test_index])
                break
    return [gsibdf_train, gsibdf_test]


def dt_train(labels, features, weight, rs=42):
    estimator = DecisionTreeClassifier(max_depth=30, min_samples_leaf=5, random_state=42)
    est = estimator.fit(features, labels, sample_weight=weight)
    return est

def kfold_train_test(sib, nonsib):
    kfolds = 10
    stats_train_error = np.empty((10,5), dtype=float)
    stats_test_error = np.empty((10,5), dtype=float)
    graphs = []
    for i in range(10):
        print("Round {}".format(i))
        # pick proportionally from each group
        train_sib, test_sib = split_stratified_groups(sib, 10, i)
        # create, select, and mix matching nonsibs
        train_nonsib = match_nonsibs(train_sib, nonsib)
        test_nonsib = match_nonsibs(test_sib, nonsib)
        train = mix_sib_nonsib(train_sib,train_nonsib, "all")
        # prune NaNs out
        train, train_prune_lbl, train_prune_prd = prune_data_for_ml(train)
        test = mix_sib_nonsib(test_sib,test_nonsib, "all")
        test, test_prune_lbl, test_prune_prd = prune_data_for_ml(test)
        # split out features, labels, and weights
        train_lbl, train_ftr = make_labels_features(train)
        test_lbl, test_ftr = make_labels_features(test)
        train_weight = get_sample_weight_one_input(train)
        test_weight = get_sample_weight_one_input(test)
        # train estimator
        est = dt_train(train_lbl, train_ftr, train_weight)   
        mcc = matthews_corrcoef(train_lbl, est.predict(train_ftr), sample_weight=train_weight)
        statsv = list(stats(train_lbl, est.predict(train_ftr)))
        statsv.append(mcc)
        stats_train_error[i] = statsv
        #print("test error: mcc of {}".format(mcc))
        mcc = matthews_corrcoef(test_lbl, est.predict(test_ftr), sample_weight=test_weight)
        statsv = list(stats(test_lbl, est.predict(test_ftr)))
        statsv.append(mcc)
        stats_test_error[i] = statsv
        #stats_test_error[i]  =  stats(test_lbl, est.predict(test_ftr))
        graph = dt_plot(est, train_ftr)
        graphs.append(graph)
        #Image(graph.create_png())  
    return stats_train_error, stats_test_error, graphs

## Evaluate Hand-Tuned Algo First

1. Calculate Training Error
1. Evaluate only new hosts to get Test error

### Training Error

#### Numbers for algo used in Row 3 of Table II in paper

In [5]:
for i in range(1,2):
    print("############# Round {} ##############".format(i))
    sib, nonsib = get_pd_files("../../../gt{}/".format(i))
    #print("Columns: {}".format(list(sib.columns.values)))
    get_ouralgo_stats(sib, nonsib)


############# Round 1 ##############
Loading from filenames ../../../gt1/hosts.csvcapture.pcap.ts.siblingresult.csv and ../../../gt1/hosts.csv__nonsiblings_seed1_n679capture.pcap.ts.siblingresult.csv
Read 279 siblings and 82026 non-siblings from files.
Our algo: Not deciding on 70 pairs for unknown/error reasons.
Found 261 sibs and 81974 nonsibs, weights: 0.9968 and 0.0032, #weights: 82235
Our algo stats: (70) undecided, mcc: 0.988402745932266, f1: 0.9883720930232558
Correct: 82229, incorrect 6, TP 255, FP 0, TN 81974, FN6, Prec. 100.0, Rec. 97.7, Spec. 100.0, Acc. 99.99%


### Test Error

In [11]:
runs = 8
hlostats = np.zeros((runs-1,5), dtype=float)
hlbstats = np.zeros((runs-1,5), dtype=float)
gsdo =  dict() # group stats dict
gsdb =  dict() # group stats dict
mlstatsd_tee = dict()
mlstatsd_tre = dict()
graphs = []
for i in range(2,runs):
    print("############# Round {} ##############".format(i))
    sib, nonsib = get_pd_files("../../../algo-eval/gt{}/".format(i)) ## folder algo-eval makes this the magic!
    # high level
    hlostats[i - 1] = tuple(get_ouralgo_stats(sib, nonsib))
    hlbstats[i - 1] = get_bev_stats(sib, nonsib)
    
    # group-level
    groups = assign_groups(sib)
    groupset = set(groups)
    for j in groupset:
        if j not in gsdo:
            gsdo[j] = np.zeros((runs,5), dtype=float)
            gsdb[j] = np.zeros((runs,5), dtype=float)
        print("## GROUP: {}".format(j))
        groupsib = sib[sib["group"] == j].copy()
        groupnonsib = match_nonsibs(groupsib, nonsib)
        gsdo[j][i-1] = get_ouralgo_stats(groupsib, groupnonsib)
        gsdb[j][i-1] = get_bev_stats(groupsib, groupnonsib)
    # decision-tree
    #mlstatsd_tre[str(i)+"_tre"], mlstatsd_tee[str(i)+"_tee"], graph = kfold_train_test(sib, nonsib) # returns 2 sets of 10x4 arrays
    #graphs.append(graph)

############# Round 2 ##############
Loading from filenames ../../../algo-eval/gt2/hosts.csvcapture.pcap.ts.siblingresult.csv and ../../../algo-eval/gt2/hosts.csv__nonsiblings_seed1_n407_capture.pcap.ts.siblingresult.csv
Read 369 siblings and 145041 non-siblings from files.
Our algo: Not deciding on 32 pairs for unknown/error reasons.
Found 339 sibs and 145039 nonsibs, weights: 0.9977 and 0.0023, #weights: 145378
Our algo stats: (32) undecided, mcc: 0.9806123557438454, f1: 0.9805680119581464
Correct: 145365, incorrect 13, TP 328, FP 2, TN 145037, FN11, Prec. 99.39, Rec. 96.76, Spec. 100.0, Acc. 99.99%
Found 367 sibs and 145039 nonsibs, weights: 0.9975 and 0.0025, #weights: 145406
Beverly algo: Not deciding on 4 pairs for NaN and 0 pairs for unknown/error reasons.
Beverly algo stats: (0) undecided, mcc: 0.08428963703551832, f1: 0.01924351260370557
Correct: 108405, incorrect 37001, TP 363, FP 36997, TN 108042, FN4, Prec. 0.97, Rec. 98.91, Spec. 74.49, Acc. 74.55%
## GROUP: servers
Genera

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:516: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(var_yt * var_yp)


Read 370 siblings and 145804 non-siblings from files.
Our algo: Not deciding on 44 pairs for unknown/error reasons.
Found 332 sibs and 145798 nonsibs, weights: 0.9977 and 0.0023, #weights: 146130
Our algo stats: (44) undecided, mcc: 0.9740231486636811, f1: 0.9738863287250383
Correct: 146113, incorrect 17, TP 317, FP 2, TN 145796, FN15, Prec. 99.37, Rec. 95.48, Spec. 100.0, Acc. 99.99%
Found 370 sibs and 145798 nonsibs, weights: 0.9975 and 0.0025, #weights: 146168
Beverly algo: Not deciding on 6 pairs for NaN and 0 pairs for unknown/error reasons.
Beverly algo stats: (0) undecided, mcc: 0.08416894811169548, f1: 0.01921612894757567
Correct: 108807, incorrect 37361, TP 366, FP 37357, TN 108441, FN4, Prec. 0.97, Rec. 98.92, Spec. 74.38, Acc. 74.44%
## GROUP: servers
Generated 210 non-sibling candidates from 15 siblings.
Our algo: Not deciding on 1 pairs for unknown/error reasons.
Found 14 sibs and 210 nonsibs, weights: 0.9375 and 0.0625, #weights: 224
Our algo stats: (1) undecided, mcc: 0.

#### Evaluation number of our algo is used for row 4 in Table II in the paper

In [17]:
# cleanse rows with all zeros (something went wrong there)
hlbstats = hlbstats[~np.all(hlbstats == 0, axis=1)]
mean_prec = round(np.mean(hlbstats[:,0]),2)
mean_mcc = round(np.mean(hlbstats[:,4]),2)
print("High-Level Beverly stats against 2016 gt, mean across all measurements: Precision {}%, MCC {}".format(
    mean_prec, mean_mcc))
hlostats = hlostats[~np.all(hlostats == 0, axis=1)]
mean_prec = round(np.mean(hlostats[:,0]),2)
mean_mcc = round(np.mean(hlostats[:,4]),2)
print("High-Level Algo stats against 2016-12 gt (excluding 2016-03 gt), mean across all measurements: Precision {}%, MCC {}".format(
    mean_prec, mean_mcc))
hlostats[:,[0,4]]

High-Level Beverly stats against 2016 gt, mean across all measurements: Precision 0.97%, MCC 0.08
High-Level Algo stats against 2016-12 gt (excluding 2016-03 gt), mean across all measurements: Precision 99.49%, MCC 0.98


array([[  99.39      ,    0.98061236],
       [  99.37      ,    0.97402315],
       [  99.41      ,    0.97397716],
       [ 100.        ,    0.98389132],
       [  99.39      ,    0.98037884],
       [  99.37      ,    0.97977162]])

## Evaluation of Hand-Tuned Algo for subgroups (lower part of Table III in paper)

In [19]:
for group in set(gsdo.keys()):
    x = gsdo[group]
    x = x[~np.all(x == 0, axis=1)]
    mean_prec = round(np.mean(x[0:,0]),2)
    mean_mcc = round(np.mean(x[0:,4]),2)
    print("Algo group stats against 2016-12 gt (excluding 2016-03 gt), mean across all measurements: {}, Precision {}%, MCC {}".format(
    group, mean_prec, mean_mcc))

Algo group stats against 2016-12 gt (excluding 2016-03 gt), mean across all measurements: servers, Precision 100.0%, MCC 0.99
Algo group stats against 2016-12 gt (excluding 2016-03 gt), mean across all measurements: RAv2, Precision 99.16%, MCC 0.98
Algo group stats against 2016-12 gt (excluding 2016-03 gt), mean across all measurements: nlnog, Precision 100.0%, MCC 0.98
Algo group stats against 2016-12 gt (excluding 2016-03 gt), mean across all measurements: RAv1, Precision 100.0%, MCC 1.0
